In [3]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, utils
import torchvision.utils as vutils
import matplotlib.pyplot as plt
import numpy as np

In [4]:
BATCH_SIZE = 128
EPOCH = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
#학습 데이터 준비
transform = transforms.Compose([
    transforms.Resize(64),
    transforms.CenterCrop(64), #얼굴이므로 가운데가 좋을 것
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

dataset = datasets.ImageFolder(root = './data', transform = transform)
dataloader = DataLoader(dataset, batch_size = BATCH_SIZE, shuffle = True)

In [15]:
#Generator(모조 화가) 클래스
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            #역합성곱 연산을 하여 입력의 크기를 늘려준다. 최종목표는 3*64*64이다.
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias=False),  #100*1*1 -> 512*4*4
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False), #512*4*4 -> 256*8*8
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False), #256*8*8 -> 128*16*16
            nn.BatchNorm2d(128),
            nn.ReLU(True),
           
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False), #128*16*16 -> 64*32*32
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False), #64*32*32 -> 3*64*64 
            nn.Tanh()     
        )

    def forward(self, input):
        return self.main(input)

In [16]:
#Discriminator(감별사) 클래스
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            #합성곱 연산을 하여 크기를 줄여준다 최종목표는 1*1*1이다.
            nn.Conv2d(3, 64, 4, 2, 1, bias=False), #3*64*64 -> 64*32*32
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(64, 128, 4, 2, 1, bias=False), #64*32*32 -> 128*16*16
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(128, 256, 4, 2, 1, bias=False), #128*16*16 -> 256*8*8
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(256, 512, 4, 2, 1, bias=False), #256*8*8 -> 512*4*4
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(512, 1, 4, 1, 0, bias=False), #512*4*4 -> 1*1*1
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input).view(-1, 1).squeeze(1)

In [8]:
#가중치 초기화 함수
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [9]:
#이미지 출력 함수
def show_generated_images(images, num_images=64):
    plt.figure(figsize=(10, 10))
    plt.axis("off")
    plt.title("Generated Images")
    images = vutils.make_grid(images[:num_images], padding=2, normalize=True)
    images = np.transpose(images.cpu(), (1, 2, 0))
    plt.imshow(images)
    plt.show()

#이미지 저장 함수
def save_generated_images(images, num_images, epoch, idx):
    plt.figure(figsize=(10, 10))
    plt.axis("off")
    plt.title("Generated Images")
    images = vutils.make_grid(images[:num_images], padding=2, normalize=True)
    images = np.transpose(images.cpu(), (1, 2, 0))
    # plt.imshow(images)
    fname = './output/image_'+str(epoch)+'_'+str(idx)+'.jpg'
    plt.imsave(fname, images.numpy())
    plt.close()

In [17]:
#모델 생성
netG = Generator() 
netD = Discriminator()

In [18]:
#가중치 초기화
netG.apply(weights_init)
netD.apply(weights_init)

Discriminator(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

In [19]:
netG.to(device)
netD.to(device)

criterion = nn.BCELoss() #Binary Cross Entropy
optimizerD = optim.Adam(netD.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=0.0002, betas=(0.5, 0.999))

fixed_noise = torch.randn(64, 100, 1, 1, device = device) #이미지 생성 중간 점검에 사용됨

In [23]:
for epoch in range(EPOCH):
    for i, data in enumerate(dataloader, 0):
        #Discriminator 먼저
        netD.zero_grad()
        real_data = data[0].to(device) #실제 사진
        batch_size = real_data.size(0) 
        real_label = torch.full((batch_size,), 1, dtype=torch.float, device = device) #정답은 1
        fake_label = torch.full((batch_size,), 0, dtype=torch.float, device = device) #정답은 0

        output = netD(real_data).view(-1) #순전파로 진짜일 확률 계산
        errD_real = criterion(output, real_label)
        errD_real.backward() #역전파로 그라디언트 계산

        rand_input = torch.randn(batch_size, 100, 1, 1, device=device) #100*1 크기의 랜덤 tensor
        fake_data = netG(rand_input)
        output = netD(fake_data.detach()).view(-1) #가짜사진이 진짜라고 생각하는 확률

        errD_fake = criterion(output, fake_label)
        errD_fake.backward() #역전파로 그라디언트 계산

        errD = errD_real + errD_fake #이 둘을 합친것이 Discriminator의 error
        optimizerD.step() #갱신

        #Generator도 학습
        netG.zero_grad()
        output = netD(fake_data).view(-1) #가짜사진이 진짜라고 생각하는 확률
        errG = criterion(output, real_label) #ouput이 1이 높을 수록 그럴듯하게 잘 만든것 -> 손실이 작음
        errG.backward()
        optimizerG.step() #갱신

        if i % 50 == 0:
            print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f'
                  % (epoch, EPOCH, i, len(dataloader), errD.item(), errG.item()))
            fake_images = netG(fixed_noise)
            save_generated_images(fake_images, 64, epoch=epoch, idx=i)

[0/10][0/1583] Loss_D: 0.8821 Loss_G: 6.3869
[0/10][50/1583] Loss_D: 0.2535 Loss_G: 27.6500
[0/10][100/1583] Loss_D: 0.4633 Loss_G: 9.5946
[0/10][150/1583] Loss_D: 0.5024 Loss_G: 4.5657
[0/10][200/1583] Loss_D: 0.3604 Loss_G: 3.9084
[0/10][250/1583] Loss_D: 0.3524 Loss_G: 5.8282
[0/10][300/1583] Loss_D: 0.6376 Loss_G: 5.0437
[0/10][350/1583] Loss_D: 0.5566 Loss_G: 2.7458
[0/10][400/1583] Loss_D: 0.3889 Loss_G: 4.5326
[0/10][450/1583] Loss_D: 1.5843 Loss_G: 2.4811
[0/10][500/1583] Loss_D: 0.5588 Loss_G: 3.7698
[0/10][550/1583] Loss_D: 0.4487 Loss_G: 5.1489
[0/10][600/1583] Loss_D: 0.9414 Loss_G: 5.8112
[0/10][650/1583] Loss_D: 0.4494 Loss_G: 4.5196
[0/10][700/1583] Loss_D: 1.1208 Loss_G: 7.9532
[0/10][750/1583] Loss_D: 0.5686 Loss_G: 2.8373
[0/10][800/1583] Loss_D: 0.2343 Loss_G: 4.9792
[0/10][850/1583] Loss_D: 0.5722 Loss_G: 3.3445
[0/10][900/1583] Loss_D: 0.4373 Loss_G: 4.3632
[0/10][950/1583] Loss_D: 0.7310 Loss_G: 6.2940
[0/10][1000/1583] Loss_D: 1.3751 Loss_G: 4.1279
[0/10][1050/15

In [ ]:
#시각화
fake_images = netG(fixed_noise)
show_generated_images(fake_images)

In [ ]:
torch.__version__

'2.3.1+cu118'

In [ ]:
np.__version__

'2.0.1'

In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
